In [1]:
pip install konlpy 

   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   --------- ------------------------------ 4.7/19.4 MB 23.5 MB/s eta 0:00:01
   ------------------- -------------------- 9.4/19.4 MB 23.2 MB/s eta 0:00:01
   ----------------------------- ---------- 14.2/19.4 MB 23.6 MB/s eta 0:00:01
   ---------------------------------------  19.1/19.4 MB 23.2 MB/s eta 0:00:01
   ---------------------------------------- 19.4/19.4 MB 22.7 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [konlpy]
   ---------------------------------------- 2/2 [konlpy]

Note: you may need to restart the kernel to use updated packages.


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options #셀레니움에 웹드라이브에 크롬을 통해서 내부적인 옵션이 설정 되어있어야 함 class!
from bs4 import BeautifulSoup # 파싱을 위한 뷰티풀슙~
from openpyxl import Workbook
from konlpy.tag import Okt   #코리안 내츄럴 파이썬. 오픈 코리안 텍스트
from collections import Counter # 수집하고 숫자 카운트? 리스트형식
import time #싱글페이지 앱일 경우엔 js로 만드는데 너무 짧은 시간에 페이지로 가면 데이터를 찾아오지 않은 상태에서 랜딩하기 떄문 -> 그걸 방지하기 위해~

options= Options() # 옵션즈로부터 전달받은 인자값을 사용할 수 있다
options.add_argument("--no-sandbox") # 방해하지마! 모래상자개념 => 내부에서 보안정책 시행되게끔 한게 샌드박스
options.add_argument("--disable-dev-shm-usage") # 어디에 넣어놓고 다시 찾아올 수 있음, 개발자도구안에 가상메모리 공간에 저장 못할 수 있기 떄문에 우리의 메모리공간을 써라 not 브라우저 메모리공간!
options.add_argument("--headless") # 새창 안뜨고 몰래 값 도출!

driver = webdriver.Chrome(options=options) # 위에 옵션값을 가지고 있는 브라우저를 통해 컬리에 들어가야 한다

url = "https://www.kurly.com/goods/1000338161"
test = driver.get(url)
time.sleep(5)

soup = BeautifulSoup(driver.page_source, "html.parser")
reviews = soup.select("p.css-y49dcn.e36z05c13")
review_lists = [review.get_text().strip() for review in reviews] #태그가 필요 없기 떄문에

#print(review_lists)
driver.quit() # 다 하면 나가자! 

wb = Workbook()
ws = wb.active
ws.title = "kurly Reviews"
ws.append(["리뷰내용"])

for item in review_lists :
    ws.append([item])

okt = Okt()
all_text = " ".join(review_lists)
nouns = okt.nouns(all_text) #명사만 찾아오세요
filtered_nouns = [n for n in nouns if len(n) > 1]
#print(filtered_nouns)
count = Counter(filtered_nouns)

ws_keywords = wb.create_sheet("Keyword Frequency")
ws_keywords.append(["키워드", "빈도수"])
for word, freq in count.most_common(30) : # 위로부터 () 만큼 카운트  / 단어와 빈도수
    ws_keywords.append([word, freq])

wb.save("kurly_reviews.xlsx")
print("엑셀 저장 완료 : kurly_reviews.xlsx")

엑셀 저장 완료 : kurly_reviews.xlsx


In [13]:
from openpyxl import load_workbook # 기존 엑셀 파일 불러옴
from konlpy.tag import Okt
from collections import Counter

wb = load_workbook("연관키워드 20250722 1448.xlsx")
ws = wb.active 

texts = [row[0]for row in ws.iter_rows(min_row = 2, max_col = 1, values_only= True) if row [0]] # row에 값이 있다면-> 답 for (어떠한 닶) if ~하면 

okt = Okt()
all_text = " ".join(texts) # 리스트였던 애들이 통으로 도출됨
nouns = okt.nouns(all_text)
filtered_nouns = [n for n in nouns if len(n) > 1]
count = Counter(filtered_nouns)

ws_keywords = wb.create_sheet("키워드 전체분석")
ws_keywords.append(["명사", "빈도수"])
for word, freq in sorted(count.items(), key=lambda x : x[1], reverse = True): # 내림차순 정렬, lambda 익명함수 x에 값을 넣어라..?ㅋ 트루면 내림 펄스는 올림
    ws_keywords.append([word,freq])

wb.save("연관키워드_분석결과.xlsx")
print("완료 : 전체 키워드 분석 결과 저장됨")

완료 : 전체 키워드 분석 결과 저장됨


In [22]:
from openpyxl import load_workbook
from konlpy.tag import Okt
from collections import Counter

wb = load_workbook("연관키워드 20250722 1551.xlsx")
ws = wb.active

texts = [row[0]for row in ws.iter_rows(min_row = 2, max_col= 1, values_only= True) if row [0]]

okt = Okt()
all_text = " ".join(texts)
nouns = okt.nouns(all_text)
filtered_nouns = [n for n in nouns if len(n)>1]
count = Counter(filtered_nouns)

ws_keywords = wb.create_sheet("키워드 전체분석")
ws_keywords.append(["명사", "빈도수"])
for word, freq in sorted(count.items(), key=lambda x:x[1], reverse = True) :
    ws_keywords.append([word,freq])

wb.save("유산균연관키워드_분석결고.xlsx")
print("완료 : 전체 키워드 분석 결과 저장됨")

완료 : 전체 키워드 분석 결과 저장됨


In [23]:
from openpyxl import load_workbook # 기존 엑셀 파일 불러옴
from konlpy.tag import Okt
from collections import Counter
import re

wb = load_workbook("연관키워드 20250722 1448.xlsx")
ws = wb.active 

texts = [row[0]for row in ws.iter_rows(min_row = 2, max_col = 1, values_only= True) if row [0]] # row에 값이 있다면-> 답 for (어떠한 닶) if ~하면 

okt = Okt()
all_text = " ".join(texts) # 리스트였던 애들이 통으로 도출됨

nouns_tokens = okt.nouns(all_text)

regex_tokens = re.findall(r'[A-Za-z]*\d+[A-Za-z]+|\d+[A-Za-z]*', all_text) #(,찾아와야하는 대상) r 로우스트링 있을수도 없을수도 있음~ * \d는 숫자, | 버티컬바는 또는

valid_one_letter = {"금", "은", "동"}
stopwords = {"사용", "정도", "조금", "제품", "때문", "생각", "만족", "구매", "다음"}

combined_tokens = nouns_tokens + regex_tokens
filtered_tokens = [ t for t in combined_tokens if (len(t)>1 or t in valid_one_letter) and t not in stopwords]

count = Counter(filtered_tokens)

ws_keywords = wb.create_sheet("키워드 전체분석")
ws_keywords.append(["명사", "빈도수"])
for word, freq in sorted(count.items(), key=lambda x : x[1], reverse = True): # 내림차순 정렬, lambda 익명함수 x에 값을 넣어라..?ㅋ 트루면 내림 펄스는 올림
    ws_keywords.append([word,freq])

wb.save("연관키워드_분석결과.xlsx")
print("완료 : 전체 키워드 분석 결과 저장됨")

완료 : 전체 키워드 분석 결과 저장됨
